In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
denver_path = "./denvercrime.csv"
crime_code_path = "./offense_codes.csv"
holidayDataDenver = pd.read_csv('Denver_Holiday.csv')
holidayDataVancouver = pd.read_csv('Vancouver_Holiday.csv')
eventDataDenver = pd.read_csv('Denver_events.csv')
df = pd.read_csv(denver_path)[:10]
crime_code_df = pd.read_csv(crime_code_path)

In [ ]:
crime_code_index = []
crime_details=[]
for index, row in crime_code_df.iterrows():
    crime_code_index.append((row["OFFENSE_CODE"],row["OFFENSE_CODE_EXTENSION"]))
    crime_details.append(row["OFFENSE_TYPE_NAME"])

In [ ]:
def time_cov(raw_in):
    if type(raw_in)==float:
        return raw_in,raw_in,raw_in,raw_in,raw_in,raw_in
    datetime_raw = datetime.strptime(raw_in, '%m/%d/%Y %I:%M:%S %p')
    date = int(datetime_raw.strftime("%Y%m%d"))
    hour = datetime_raw.hour
    minute = datetime_raw.minute
    second = datetime_raw.second
    dow = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"][datetime_raw.weekday()]
    return date,hour,minute,second,dow,datetime_raw


In [ ]:
fact_df = pd.DataFrame(columns=['location_Key','crime_key', 'date_key','event_key', 'is_traffic','is_fatal','is_nighttime'])
location_df = pd.DataFrame(columns=['location_Key','city','location_name', 'neighborhood','latitude','longitude', 'crime_rate'])
crime_df= pd.DataFrame(columns=['crime_key','crime_category_name', 'crime_type_name','crime_details',
                                 'start_hour', 'start_minute',"start_second",
                                 'end_hour', 'end_minute',"end_second",
                                 'report_hour', 'report_minute',"report_second"])
#no need to create twice in Vancouver
date_df = pd.DataFrame(columns=['Date_Key', 'Year','Month','Day', 'Day_of_Week','Is_Weekend','Is_Holiday_Vancouver','Holiday_Name_Vancouver','Is_Holiday_Denver','Holiday_Name_Denver'])
event_df = pd.DataFrame(columns=['event_key','event_name','event_type','event_location','event_neighbourhood','event_city','event_start_date','event_end_date','event_location_size'])


In [ ]:
#Full Date Dimesion
start='20150101'
end='20191231'
s = pd.Series(pd.date_range(start, end))
date_df["Date_Key"] = s.dt.strftime('%Y%m%d')
date_df["Day_of_Week"] = s.dt.weekday_name
date_df['Is_Weekend'] = ((pd.to_numeric( s.dt.weekday)) // 5 == 1).astype('int64')
date_df["Year"] = s.dt.year
date_df["Month"] = s.dt.month
date_df["Day"] = s.dt.day
date_df["Is_Holiday_Denver"] = 0
date_df["Is_Holiday_Vancouver"] = 0
#Put Denver Holiday csv into Date Dimension
for idx_hol, row_hol in holidayDataDenver.iterrows():
    for idx_date, row_date in date_df.iterrows():
        if row_hol['YEAR']==row_date['Year'] and row_hol['MONTH']==row_date['Month'] and row_hol['DAY']== row_date['Day']:
            date_df.at[idx_date, 'Is_Holiday_Denver'] = 1
            date_df.at[idx_date, 'Holiday_Name_Denver'] = row_hol['NAME']
#Put Vancouver Holiday csv into Date Dimension
for idx_hol, row_hol in holidayDataVancouver.iterrows():
    for idx_date, row_date in date_df.iterrows():
        if row_hol['YEAR']==row_date['Year'] and row_hol['MONTH']==row_date['Month'] and row_hol['DAY']== row_date['Day']:
            date_df.at[idx_date, 'Is_Holiday_Vancouver'] = 1
            date_df.at[idx_date, 'Holiday_Name_Vancouver'] = row_hol['NAME']
date_df.head()

In [ ]:
#Event Dimension only has Denver Event
for idx_event, row_event in eventDataDenver.iterrows():
    event_row=[idx_event+3000000]
    event_row += [row_event['NAME'],row_event['TYPE'],row_event['LOCATION'],row_event['NEIGHBOURHOOD'],'Denver',row_event['DATE_START'],row_event['DATE_END'],row_event['LOCATION-SIZE']]
    event_df.loc[len(event_df)] = event_row
event_df.head()

In [ ]:
for index, row in df.iterrows():
    s_d,s_h,s_m,s_s,s_dow,s_dt = time_cov(row["FIRST_OCCURRENCE_DATE"])
    if not(2015<=s_dt.year<=2019):
        continue
    location_id = index + 1000000
    city = "Denver"
    location = row["INCIDENT_ADDRESS"]
    neighborhood = row["NEIGHBORHOOD_ID"]
    lat = row["GEO_LAT"]
    lon = row["GEO_LON"]
    crime_rate = 0
    location_df.loc[index] = [location_id,city,location,neighborhood,lat,lon,crime_rate]
    
    
    crime_id = index + 2000000
    crime_category_name = row["OFFENSE_CATEGORY_ID"]
    crime_type_name = row["OFFENSE_TYPE_ID"]
    crime_detail = crime_details[crime_code_index.index((row["OFFENSE_CODE"],row["OFFENSE_CODE_EXTENSION"]))]
    
    e_d,e_h,e_m,e_s,e_dow,e_dt = time_cov(row["LAST_OCCURRENCE_DATE"])
    r_d,r_h,r_m,r_s,r_dow,r_dt = time_cov(row["REPORTED_DATE"])
    crime_df.loc[index] = [crime_id,crime_category_name,crime_type_name,crime_detail,s_h,s_m,s_s,e_h,e_m,e_s,r_h,r_m,r_s]
    
    
    date_id = int(s_dt.strftime("%Y%m%d"))
    
    
    #Event
    event_id = np.nan
    for idx_event, row_event in event_df.iterrows():
        #the crime is in the event period
        if date_id>=row_event['event_start_date'] and date_id<=row_event['event_end_date']:
            #the event is city size and there is no event assign to the crime
            if row_event['event_location_size'] == 'city':
                if np.isnan(event_id):
                    event_id = row_event['event_key']
            #the event is not city size
            else:
                #the neighborhood are same
                if neighborhood == row_event['event_neighbourhood']:
                    event_id = row_event['event_key']
                    
    
    
    is_night = 0 if 7 <= s_h <= 18 else 1
    
    fatal_keywords=['FATAL','DEAD','DEATH','KILL','HOMICIDE','MURDER']
    is_fatal = 0
    for keyword in fatal_keywords:
        for name in [crime_category_name,crime_type_name,crime_detail]:
            if keyword in name.upper():
                is_fatal = 1

    
    fact_df.loc[index] = [location_id,crime_id,date_id,event_id,row["IS_TRAFFIC"],is_fatal,is_night]

In [ ]:
location_df.head()

In [ ]:
crime_df.head()

In [ ]:
# fact_df convert float to int
for col in fact_df.columns:
    if col !="event_key":
        fact_df[[col]]=fact_df[[col]].astype(int)

In [ ]:
fact_df.head()

In [ ]:
df.head()